## Gathering

In [ ]:
import pandas as pd
import json
import requests
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# load twitter_archive_enhanced into a dataframe,tae stands for twitter archive enhanced
tae=pd.read_csv('twitter-archive-enhanced.csv')
url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/数据清洗/WeRateDogs项目/image-predictions.tsv'
response = requests.get(url)

#open a file called image-predictions.tsv, and write response in this file
with open(url.split('/')[-1],mode='wb') as file:
    file.write(response.content)
    
# ip stands for image prediction
ip = pd.read_csv('image-predictions.tsv',sep='\t')
tweet_json = []

#read tweet_json file, and put content in a dataframe
with open('tweet_json.txt', 'r') as f:
    for line in f.readlines():
        x = json.loads(line)
        tweet_id=x['id']
        retweet_count = x['retweet_count']
        favorite_count = x['favorite_count']
        tweet_json.append({'tweet_id':tweet_id,
                          'retweet_count':retweet_count,
                          'favorite_count':favorite_count})
print(tweet_json)
# turn into a dataframe
tweet = pd.DataFrame(tweet_json, columns=['tweet_id','retweet_count','favorite_count'])


## Assessing

#### Assessing Twitter archive enhanced

In [ ]:
tae.info() # 181 retweet_id
tae.sample(20)

In [ ]:
tae['name'].value_counts()

In [ ]:
tae.rating_denominator.value_counts()
ip.info()

In [ ]:
tweet.info()

#### assessing image predictions

In [ ]:
ip.sample(30)

In [ ]:
ip.info()

In [ ]:
ip['p1_dog'].value_counts()

In [ ]:
# test the p2 results
(ip.p2_dog == False).sum()

In [ ]:
# test the p3 results
(ip.p3_dog == False).sum()

In [ ]:
# test the result of all three pictures are not dogs
ip[(ip['p1_dog']==False) & (ip['p2_dog']==False) & (ip['p3_dog']==False)].count()

# Quality

#### data type of "time stamp"should be datetime
#### in_reply_to_status_id and in_reply_to_user_id have many missing values
#### twitter_archive_enhanced, 181 rows belong to retweet
#### doggo，floofer ，pupper，puppo have missing values
#### twitter_archive_enhanced, change non-sense dogs names to none.
#### image prediction:get rid of pics that are not dogs（p1,p2,p3 are all false）
#### image: unstandarized naming for dogs, namely: some are lower case, some have first letters of the name that are in upper case, underscore and space are used interchangeably. 
#### some denominators could be divided by 10，and some numerators can also divided by N


# Tidiness
#### can combine the statuse column
####  all three data sets are collected based on tweet_id, so we can combine these three data sets.

# Cleaning

In [ ]:
# make copies of original dataframes
tae_clean = tae.copy()
ip_clean = ip.copy()
tweet_clean = tweet.copy()

## Missing Data

### Issue 1

<p> we only care about original tweets，so we exclude the row of 'retweeted_status_x',and data related to in reply</p>


### Code

In [ ]:
#delete retweet status，as we want to focus on the original tweets
tae_clean.drop(tae_clean[tae_clean.retweeted_status_id.notnull()].index, inplace=True)


In [ ]:
#drop retweets , in_reply_to_status_id , in_reply_to_user_id
tae_clean.drop(['retweeted_status_id',
                     'retweeted_status_user_id',
                     'retweeted_status_timestamp',
                     'in_reply_to_status_id',
                     'in_reply_to_user_id'], axis=1, inplace=True)



### Testing

In [ ]:
tae_clean.info()

### Issue 2
<p> doggo，floofer ，pupper，puppo have missing values </p>
<p> combine doggo status column </p>
<p> change the naming convention of doggo to first letter in upper case </p>
<p> combine 'twitter archive enhanced'and 'tweet'two dataframes,inner join twitter id  </p>

### Code

In [ ]:

# change stage "none" to blank
tae_clean.doggo.replace('None', '', inplace=True)
tae_clean.floofer.replace('None', '', inplace=True)
tae_clean.pupper.replace('None', '', inplace=True)
tae_clean.puppo.replace('None', '', inplace=True)
tae_clean.sample(10)


In [ ]:
#combine all stages to a single column called 'stage'
tae_clean['dog_stage'] = tae_clean['doggo'] + tae_clean['floofer']+ tae_clean['pupper']+tae_clean['puppo']

tae_clean.dog_stage.value_counts()

In [ ]:
# test
tae_clean.info()

In [ ]:
# Rename values in dog_stage column
tae_clean.loc[tae_clean.dog_stage == 'pupper', 'dog_stage'] = 'Pupper'
tae_clean.loc[tae_clean.dog_stage == 'doggo', 'dog_stage'] = 'Doggo'
tae_clean.loc[tae_clean.dog_stage == 'puppo', 'dog_stage'] = 'Puppo'
tae_clean.loc[tae_clean.dog_stage == 'doggopupper', 'dog_stage']= 'Doggo, Pupper'
tae_clean.loc[tae_clean.dog_stage == 'floofer', 'dog_stage'] ='Floofer'
tae_clean.loc[tae_clean.dog_stage == 'doggopuppo', 'dog_stage']= 'Doggo, Puppo'
tae_clean.loc[tae_clean.dog_stage == 'doggofloofer', 'dog_stage'] = 'Doggo, Floofer'

In [ ]:
# Replace blank cells with NaNs
tae_clean.loc[tae_clean.dog_stage == '', 'dog_stage'] = np.nan

In [ ]:
# Replace NaNs with text so we have non-null values
tae_clean.dog_stage = tae_clean.dog_stage.fillna('Unknown')

In [ ]:
# Drop unnecessary columns
tae_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

In [ ]:
#Join the image_clean df to the twitter_archive_master df
twitter_archive_master = pd.merge(tae_clean,tweet_clean,on='tweet_id')

### Testing

In [ ]:
# test
twitter_archive_master.info()

In [ ]:
twitter_archive_master.head(10)

### Issue 3
<p> there are pics that are not doggo</p>
<p>some dogs names have no meaning</p>

### Code

In [ ]:
# Create a new column for the dog prediction summary in image_prediction
# first convert p1_dog, p2_dog, p3_dog to an integer 
prediction_summary = ['p1_dog', 'p2_dog', 'p3_dog']
for p in prediction_summary:
     ip_clean[p] = ip_clean[p].astype(int)

In [ ]:
# Create a new column that adds the total number of True and False for the 3 predictions
ip_clean['prediction'] = ip_clean.p1_dog + ip_clean.p2_dog + ip_clean.p3_dog

In [ ]:
# Replace the number with a defining text string
ip_clean['prediction'] = ip_clean['prediction'].replace(3, 'Dog')
ip_clean['prediction'] = ip_clean['prediction'].replace(2, 'Maybe Dog')
ip_clean['prediction'] = ip_clean['prediction'].replace(1, 'Maybe Dog')
ip_clean['prediction'] = ip_clean['prediction'].replace(0, 'Not Dog')
twitter_archive_master['name'].value_counts()

### Testing

In [ ]:
#test
ip_clean[['p1_dog', 'p2_dog', 'p3_dog','prediction']].sample(10)

In [ ]:
# convert non-names to None
twitter_archive_master['name'].str.lower()

mask = (twitter_archive_master.name.str.islower())|(twitter_archive_master.name == 'None')

twitter_archive_master.loc[mask, 'name'] = 'No_name'


### Testing

In [ ]:
twitter_archive_master['name'].value_counts()

### Issue 4
<p> combine 'twitter archive enhanced', 'tweet', 'image prediction', these three dataframes，inner join twitter id </p>

### Code

In [ ]:
#Join the image_clean df to the twitter_archive_master df 
twitter_archive_master = pd.merge(twitter_archive_master, ip_clean, on='tweet_id')

### Testing

In [ ]:
#test
twitter_archive_master[['p1', 'p2', 'p3']].sample(10)

### Issue 5
<p> get rid of rows that are doggo </p>
<p> get rid of underscores</p>
<p> change 'timestamp' to 'datetime'</p>

### Code

In [ ]:
#Remove non-dogs from master dataset
twitter_archive_master = twitter_archive_master[twitter_archive_master['prediction'] != "Not Dog"]

In [ ]:
# For predictions (p1, p2, p3), remove underscores and make title case.
predictions = ['p1', 'p2', 'p3']
for p in predictions:
     twitter_archive_master[p] = twitter_archive_master[p].str.title().str.replace('_', " ")

In [ ]:
#change the type of timestamp to datetime
twitter_archive_master['timestamp'] = pd.to_datetime(twitter_archive_master['timestamp'])


### Testing

In [ ]:
twitter_archive_master.info()
twitter_archive_master.head(5)

### Issue 6
<p>the majority of denominators are 10, and some denominators are multiples of 10</p>

### Code

In [ ]:
# make a rating column where the numerator is divided by denominator
twitter_archive_master['rating'] = twitter_archive_master['rating_numerator']/twitter_archive_master['rating_denominator']

### Testing

In [ ]:
twitter_archive_master.info()
twitter_archive_master.sample(10)

In [ ]:
twitter_archive_master.to_csv('twitter_archive_master.csv')

In [ ]:
#plot the relationship between favorite count and retweets
sns.regplot(x="retweet_count", y="favorite_count", data=twitter_archive_master, scatter_kws={'alpha':0.2})
plt.title('Retweet v. Favorite Count', size=16)
plt.xlabel('Retweets', size=12)
plt.ylabel('Favorites', size=12)
plt.savefig('retweet-favorite.png');



In [ ]:
# the most common stage of dogs
twitter_archive_master['dog_stage'].value_counts()

In [ ]:
ip_clean['prediction'].value_counts()

# conclusion

#### there is a positive correlation between the favourites and retweets.
#### pupper stage is the most common stage
#### about 60% of the images are actually dogs